In [1]:
%load_ext autoreload
%autoreload 2
import torch

import os
from os import path
import sys
from rich import print
cwd = os.getcwd().split(os.path.sep)

# point to the git repository
while cwd[-1] != "AttentionGeometry":
    os.chdir("..")
    cwd = os.getcwd().split(os.path.sep)
print(f">> current directory : {os.getcwd()}")

# add the root directory
sys.path.append(os.path.join(os.getcwd(), "src"))

# cache and data cache
cache_path = path.join(os.getcwd() ,'.cache_bert')
dataset_path = path.join(cache_path, 'dataset')
log_path = path.join(cache_path, 'logs')
model_path = path.join(cache_path, 'models')
print(f">> cache path : {cache_path}")
print(f">> model path : {model_path}")
print(f">> dataset path : {dataset_path}")
print(f">> logs path : {log_path}")
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

>> current directory : C:\Users\loicf\Documents\IRISA\TALN_2023\AttentionGeometry

>> cache path : C:\Users\loicf\Documents\IRISA\TALN_2023\AttentionGeometry\.cache_bert

>> model path : C:\Users\loicf\Documents\IRISA\TALN_2023\AttentionGeometry\.cache_bert\models

>> dataset path : C:\Users\loicf\Documents\IRISA\TALN_2023\AttentionGeometry\.cache_bert\dataset

>> logs path : C:\Users\loicf\Documents\IRISA\TALN_2023\AttentionGeometry\.cache_bert\logs

In [2]:
from datasets import load_dataset
dataset = load_dataset("yelp_polarity", split='test')

Reusing dataset yelp_polarity (C:\Users\loicf\.cache\huggingface\datasets\yelp_polarity\plain_text\1.0.0\14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61)


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [4]:
def tokenize_function(data):
    return tokenizer(data["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
#tokenized_datasets = tokenized_datasets.shuffle().select(range(100))

Loading cached processed dataset at C:\Users\loicf\.cache\huggingface\datasets\yelp_polarity\plain_text\1.0.0\14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61\cache-e98a422d6de965cc.arrow


In [5]:
print(tokenized_datasets)

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 38000
})

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    os.path.join(log_path, "yelp", "yelp-ft-3"),
    num_labels=2,
    output_hidden_states=True
)
model = model.eval()

In [ ]:
# get the r_i
import numpy as np
class_1 = 0
r_0 = Nonex
rep = None
for i in range(len(tokenized_datasets)):
    ex = tokenized_datasets[i]
    label = tokenized_datasets[i]["label"]
    ids = torch.tensor(ex["input_ids"]).unsqueeze(0)
    att_mask = attention_mask=torch.tensor(ex["attention_mask"]).unsqueeze(0)
    output_model = model(
        input_ids = ids, 
        attention_mask = att_mask
    )
    pred = output_model.logits
    if label == class_1 and np.argmax(pred.detach().cpu().numpy()) == label:
        r_0 = outm
        put_model.hidden_states[-1][0, 0, :]
        print("pred found")
        rep = tokenized_datasets[i]
        idx = i
        break

In [8]:
r_0.shape

torch.Size([768])

In [9]:
print(rep["text"])

Last summer I had an appointment to get new tires and had to wait a super long time. I also went in this week for 
them to fix a minor problem with a tire they put on. They \""fixed\"" it for free, and the very next morning I had 
the same issue. I called to complain, and the \""manager\"" didn't even apologize!!! So frustrated. Never going 
back.  They seem overpriced, too.

In [10]:
print(idx)

1

On a ici le représentant de la classe 0

In [11]:
cos_res = []
legend = []
ds = tokenized_datasets
from tqdm.notebook import tqdm
model = model.to(DEVICE)
model = model.eval()
r_0 = r_0.to(DEVICE)

labels = []
cosine = []
pos = []


with torch.no_grad():
    for i in tqdm(range(len(tokenized_datasets))):
        if i != idx:
            ex = ds[i]
            label = ex["label"]
            ids = torch.tensor(ex["input_ids"]).unsqueeze(0)
            att_mask = attention_mask=torch.tensor(ex["attention_mask"]).unsqueeze(0)
            output_model = model(
                input_ids = ids.to(DEVICE), 
                attention_mask = att_mask.to(DEVICE),
            )

            temp = output_model.hidden_states[-1][0, 0, :]
            scalar = torch.dot(r_0, temp)
            cosine.append((scalar / (torch.norm(temp) * torch.norm(r_0))).item())
            labels.append(label)
            pos.append(i)

  0%|          | 0/38000 [00:00<?, ?it/s]

Argmax avec le pivot

In [12]:
cosine = np.array(cosine)
labels = np.array(labels)
pos = np.array(pos)


mask1 = (labels == 0)
temp_labels = labels[mask1]
temp_pos = pos[mask1]
temp_cos = cosine[mask1]
p = np.argmax(temp_cos)
temp_idx = int(temp_pos[p])
print(temp_idx)
print("---")
print("cos : ", temp_cos[p])
print(tokenized_datasets[temp_idx]["text"])

mask1 = (labels == 1)
temp_labels = labels[mask1]
temp_pos = pos[mask1]
temp_cos = cosine[mask1]
p = np.argmax(temp_cos)
temp_idx = int(temp_pos[p])
print(temp_idx)
print("---")
print("cos : ", temp_cos[p])
print(tokenized_datasets[temp_idx]["text"])

14121

---

cos :  0.9637202620506287

Been coming to cafe rio for awesome fast Mexican food for years. Lived in Utah for a while...just like you Camilla 
k. I loved the ones in Utah and thought I would give this one a chance. The manager guy, don't know his name, is a 
total jerk. The food wasn't even warm. It was cold. My wife and I are sitting here right now and I'm so upset that 
I have to leave this review right now. Just awful service and not even good food anymore. Gradually getting worse. 
I'm going to costa vida from now on.

30701

---

cos :  0.9397520422935486

SO GOOD!!!!!!! The only roll I got that wasn't good was a lobster roll. It just had no flavor. Everything else I 
had was AMAZING!!! \nNow that I'm done raving about the food, I do have two complaints. 1) The hostess wasn't super
friendly or anything. She was really hard to understand and made no effort to speak more clearly so we'd know what 
she was talking about. 2) There's no where near enough tables. The place is an okay size but there's probably only 
like 10 tables? Maybe I'm remembering wrong. There's also no sushi bar, which I don't care about, but some people 
do.

argmin avec le pivot

In [14]:
mask1 = (labels == 0)
temp_labels = labels[mask1]
temp_pos = pos[mask1]
temp_cos = cosine[mask1]
p = np.argmin(temp_cos)
temp_idx = int(temp_pos[p])
print(temp_idx)
print("---")
print("cos : ", temp_cos[p])
print(tokenized_datasets[temp_idx]["text"])

# search for the closest to the median
mask1 = (labels == 1)
temp_labels = labels[mask1]
temp_pos = pos[mask1]
real_temp_cos = cosine[mask1]
temp_cos = np.abs(cosine[mask1] - 0.2)
p = np.argmin(temp_cos)
temp_idx = int(temp_pos[p])
print(temp_idx)
print("---")
print("cos : ", real_temp_cos[p])
print(tokenized_datasets[temp_idx]["text"])

11413

---

cos :  0.0886659175157547

Expensive Gringo Mexican food.  Saving grace is the setting.  Wonderful pond with ducks in the middle of the 
facility.  Go for the beauty of it, not for the \""Mexican\"" food.  They seem to cater to the Cave Creek tourist 
\""semi cowboy\"" trade.

36024

---

cos :  0.19999432563781738

I am a huge steak person. I live in LA and I've been to every fancy steakhouse in LA, most of the well known 
steakhouses in NY and Vegas. \n\nEcho and Rig may not be the best steakhouse I've ever been to, but it's up there. 
What truly impressed me was the value. The steaks and appetizers were as good as Mastro's (both Vegas and LA) yet 
it cost almost half as much. The service was as good. \n\nI've told my wife that we'll be going there every time we
go to Vegas.

In [7]:
def analysis(text):
    with torch.no_grad():
        inp = tokenizer(text, return_tensors="pt")
        output = model(**inp)
        h = output.hidden_states[-1][0, 0, :]
        pred = torch.argmax(output.logits, dim=-1).item()
    return h, pred

## Syntaxique analysis

In [12]:
sent1 = "I liked this restaurant"
sent2 = "I didn't like this restaurant"
t1, p1 = analysis(sent1)
t2, p2 = analysis(sent2)
print("class 1 : ", p1)
print("class 2 : ", p2)
print(torch.dot(t1, t2)/(torch.norm(t1, p=2)*torch.norm(t2, p=2)))

class 1 :  1

class 2 :  0

tensor(0.3985)

In [19]:
sent1 = "I found it so great"
sent2 = "I found it not so great"
t1, p1 = analysis(sent1)
t2, p2 = analysis(sent2)
print("class 1 : ", p1)
print("class 2 : ", p2)
print(torch.dot(t1, t2)/(torch.norm(t1, p=2)*torch.norm(t2, p=2)))

class 1 :  1

class 2 :  1

tensor(0.8125)

## Lexical

In [14]:
sent1 = "It was good"
sent2 = "It was awfull"
t1, p1 = analysis(sent1)
t2, p2 = analysis(sent2)
print("class 1 : ", p1)
print("class 2 : ", p2)
print(torch.dot(t1, t2)/(torch.norm(t1, p=2)*torch.norm(t2, p=2)))

class 1 :  1

class 2 :  0

tensor(0.5479)

In [15]:
sent1 = "Very good"
sent2 = "Very bad"
t1, p1 = analysis(sent1)
t2, p2 = analysis(sent2)
print("class 1 : ", p1)
print("class 2 : ", p2)
print(torch.dot(t1, t2)/(torch.norm(t1, p=2)*torch.norm(t2, p=2)))

class 1 :  1

class 2 :  0

tensor(0.5530)

In [10]:
sent1 = "This is the greatest restaurant I've ever been to"
sent2 = "This is the worst restaurant I've ever been to"
t1, p1 = analysis(sent1)
t2, p2 = analysis(sent2)
print("class 1 : ", p1)
print("class 2 : ", p2)
print(torch.dot(t1, t2)/(torch.norm(t1, p=2)*torch.norm(t2, p=2)))

class 1 :  1

class 2 :  0

tensor(0.4022)

In [16]:
sent1 = "AWESOME !"
sent2 = "Horrible !"
t1, p1 = analysis(sent1)
t2, p2 = analysis(sent2)
print("class 1 : ", p1)
print("class 2 : ", p2)
print(torch.dot(t1, t2)/(torch.norm(t1, p=2)*torch.norm(t2, p=2)))

class 1 :  1

class 2 :  0

tensor(0.6690)